In [ ]:
position_training_features={}
position_training_labels={}
position_testing_features={}
position_testing_labels={}
position_feature_scalers={}
position_label_scalers={}

for position in position_data.keys():
    
    # Generate feature-label pairs for each position
    features, labels=generate_data(position_data[position])

    # Split them into training and validation
    training_features, testing_features, training_labels, testing_labels=train_test_split(features, labels)

    # Scale the data
    feature_scaler=StandardScaler()
    feature_scaler.fit(training_features)
    position_training_features[position]=feature_scaler.transform(training_features)
    position_testing_features[position]=feature_scaler.transform(testing_features)

    label_scaler=StandardScaler()
    label_scaler.fit(training_labels.reshape(-1, 1))
    position_training_labels[position]=label_scaler.transform(training_labels.reshape(-1, 1))
    position_testing_labels[position]=label_scaler.transform(testing_labels.reshape(-1, 1))

    # Save the scalers for later
    position_feature_scalers[position]=feature_scaler
    position_label_scalers[position]=label_scaler

    print(f'\nPosition: {position}')
    print(f'Training features: {training_features.shape}')
    print(f'Training labels: {training_labels.shape}')
    print(f'Testing features: {testing_features.shape}')
    print(f'Testing labels: {testing_labels.shape}')

In [ ]:
# Loop and fit a model for each position
position_models={}
position_predictions={}

for position in position_data.keys():

    # Fit the model on the training data and make predictions for the testing data
    position_models[position]=LinearRegression().fit(position_training_features[position], position_training_labels[position])
    position_predictions[position]=position_models[position].predict(position_testing_features[position])

    # Un-scale the predictions and labels
    position_testing_labels[position]=position_label_scalers[position].inverse_transform(position_testing_labels[position])
    position_predictions[position]=position_label_scalers[position].inverse_transform(position_predictions[position])

    # Calculate RMSE
    rmse=root_mean_squared_error(position_predictions[position], position_testing_labels[position])
    print(f'{position} FanPt RMSE: {rmse}')


In [ ]:
# Set-up a 2x2 figure 
fig, axs=plt.subplots(2,2, figsize=(6,6))
axs=axs.flatten()

# Add the main title
fig.suptitle('Weekly FantPt predictions, testing data', size='large')

for ax, position in zip(axs, position_data.keys()):
    ax.set_title(f'Position: {position}')
    ax.scatter(position_testing_labels[position], position_predictions[position], color='black')
    ax.set_xlabel('True FantPt')
    ax.set_ylabel('Predicted FantPt')

plt.tight_layout()
plt.show()